In [1]:
import os
import glob
import folium
import webbrowser
import pandas as pd
from PIL import Image, ImageDraw

In [2]:
#Draw Map
map = folium.Map()
# location=[ , ], zoom_start=10, tiles=''

In [ ]:
#locations and datas from url
url = '' 
file = pd.read_csv(url)

In [ ]:
# clear the coordinates
def cleanCoordinate(coord):
    try:
        coord = ''.join(char for i, char in enumerate(coord) if char.isdigit() or (char == '.' and i == coord.find('.')))
        return float(coord)
    except ValueError:
        print(f"Invalid coordinate value: {coord}")
        return None

# extract latitude and longitude from the 'location' column
file['Latitude'] = file['location'].apply(lambda x: cleanCoordinate(x.split(',')[0].strip()))
file['Longitude'] = file['location'].apply(lambda x: cleanCoordinate(x.split(',')[1].strip()))

# filter out invalid coordinates
validData = file.dropna(subset=['Latitude', 'Longitude'])
validData = validData[(validData['Latitude'] >= -90) & (validData['Latitude'] <= 90) & (validData['Longitude'] >= -180) & (validData['Longitude'] <= 180)]

In [ ]:
# function to validate and correct color codes if your csv file has Color row
def correctColorCode(color):
    if pd.isna(color):
        return '#000000'
    color = str(color).strip()
    if len(color) == 7 and color[0] == '#':
        return color
    if len(color) == 6 and color[0] != '#':
        return '#' + color
    if len(color) > 7:
        return color[:7] if color[0] == '#' else '#' + color[:6]
    return '#000000'

In [ ]:
# default icon path in case the logo is invalid
defaultIconPath = 'defaultIcon.png'
defaultCursorSize = (64, 64)

# create a default icon if it does not exist
if not os.path.exists(defaultIconPath):
    default_icon = Image.new("RGBA", defaultCursorSize, (255, 255, 255, 0))
    draw = ImageDraw.Draw(default_icon)
    draw.ellipse([0, 0, defaultCursorSize[0] - 1, defaultCursorSize[1] - 1], fill='#000000')
    default_icon.save(defaultIconPath)

In [ ]:
# remove previously saved custom icons
for iconFile in glob.glob('combinedIcon_*.png'):
    os.remove(iconFile)

# list of logo file paths from the directory
logoDirectory = ''
logoFiles = [file for file in os.listdir(logoDirectory) if file.endswith('.png')]

# create a dictionary that maps numbers (from file names) to logo paths 
# the logo file names should be '1.png', '2.png' ...
logoPaths = {
    int(os.path.splitext(file)[0]): os.path.join(logoDirectory, file)
    for file in logoFiles
}

In [ ]:
# Add markers to the map
for i, row in validData.iterrows():
    cursorSize = (64, 64)
    cursorColor = correctColorCode(row.get('Color', '#000000'))
    cursor = Image.new("RGBA", cursorSize, (255, 255, 255, 0))
    draw = ImageDraw.Draw(cursor)
    draw.ellipse([0, 0, cursorSize[0] - 1, cursorSize[1] - 1], fill=cursorColor)
    
    no = row.get('logoNo', None) #if your csv file has logoNo (logo number code) row for each column
    logoPath = logoPaths.get(int(no), None)
    logo = None
    if logoPath and os.path.exists(logoPath):
        logo = Image.open(logoPath).convert("RGBA")
    
    if logo:
        logo_size = (48, 48)
        logo = logo.resize(logo_size)
        logo_position = ((cursorSize[0] - logo_size[0]) // 2, (cursorSize[1] - logo_size[1]) // 2)
        cursor.paste(logo, logo_position, logo)
    else:
        cursor = Image.new("RGBA", cursorSize, (255, 255, 255, 0))
        draw = ImageDraw.Draw(cursor)
        draw.ellipse([0, 0, cursorSize[0] - 1, cursorSize[1] - 1], fill=cursorColor)
    
    # save the combined icon
    combinedIconPath = f'combinedIcon_{i}.png'
    cursor.save(combinedIconPath)

    # create a custom icon
    custom_icon = folium.CustomIcon(
        icon_image=combinedIconPath,
        icon_size=(32, 32)
    )
    
    # add marker to the map with custom icon
    position = (row['Latitude'], row['Longitude'])
    folium.Marker(position, popup=row.get('Name', 'No Name'), icon=custom_icon).add_to(map)

In [ ]:
# save and open
map.save('mapName.html')
webbrowser.open('mapName.html')